In [79]:
from PIL import Image

import json
import os
import shutil
import unicodedata
import uuid
# import cv2
# import mediapipe as mp
from tqdm.auto import tqdm

In [90]:
def create_new_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)           # Removes all the subdirectories!
        print('Directory existed and was cleaned')
        os.makedirs(path)

In [82]:
folder_path_raw = '../dataset/images_raw'
folder_path = '../dataset/images'

In [97]:
def convert_to_jpg(file_path, path_to_save):
    img = Image.open(file_path)
    if img.format == 'PNG': # Conversion of png to jpg
        file_name = os.path.splitext(path_to_save)[0]
        img = img.convert("RGB")
        img.save(file_name + '.jpg', 'JPEG')
    else:
        file_name = os.path.splitext(path_to_save)[0]
        img.save(file_name + '.jpg', 'JPEG')

create_new_folder(folder_path)

for file_name in tqdm(os.listdir(folder_path_raw)):
    if file_name.lower().endswith('.png') or  file_name.lower().endswith('.jpg'):
        file_path = os.path.join(folder_path_raw, file_name)
        path_to_save = os.path.join(folder_path, file_name)
        convert_to_jpg(file_path, path_to_save)

Directory existed and was cleaned


  0%|          | 0/3741 [00:00<?, ?it/s]

100%|██████████| 3741/3741 [02:10<00:00, 28.59it/s]


In [98]:
with os.scandir(folder_path) as entries: # Used scandir instead of listdir to detect Polish characters (e.g ą,ę)
    print(len(list(entries)))

3702


In [99]:
def extract_labels(folder_path):
    labels = {}
    # /home/ant/projects/psl/dataset/images/A35.jpg
    skipped = 0
    created = 0
    labels_folder = '../dataset/labels/'
    images_path = '../dataset/images_renamed'
    
    create_new_folder(labels_folder)
    create_new_folder(images_path)
    # Searching for files in a folder
    with os.scandir(folder_path) as entries: # Used scandir instead of listdir to detect Polish characters (e.g ą,ę)
        for entry in entries:
            if entry.is_file() and entry.name.lower().endswith('.jpg'): # I want to exclude files that are not jpg
                filename = entry.name
                #print(filename)
                label = ''.join([c for c in filename.split('.')[0] if not c.isdigit()])
                
                # Skip file names starting with 'img'
                if filename.lower().startswith('img'):
                    print(f'skipping starting with img {filename}')
                    skipped += 1
                    continue
                
                # Normalize the filename to ensure proper handling of diacritical marks
                normalized_filename = unicodedata.normalize('NFC', filename)

                # Check if filename contains 'sz', 'cz', 'ch' or 'rz'
                if 'sz' in filename:
                    labels[filename] = 'sz'
                elif 'cz' in filename:
                    labels[filename] = 'cz'
                elif 'ch' in filename:
                    labels[filename] = 'ch'
                elif 'rz' in filename:
                    labels[filename] = 'rz'
                else:    
                    # Retrieving the first letter of the file name    
                    labels[normalized_filename] = label
            # else:
            #     print(f'else: {entry.name}')
                
                # Prepare the label dictionary for the current image
                image_labels = {'label': label}
                
                

                new_file_name = str(uuid.uuid4()).replace('-', '')
                shutil.copyfile(os.path.join(folder_path, filename), os.path.join(images_path, new_file_name + '.jpg'))
               # Saving the results to a JSON file
                with open(os.path.join(labels_folder, f'{new_file_name}.json'), 'w', encoding='utf-8') as json_file:
                    json.dump(image_labels, json_file,  indent=4, ensure_ascii=False)
                created += 1
            else:
                skipped += 1  
    print(created, skipped)

# Calling the function
extract_labels(folder_path)

Directory existed and was cleaned
Directory existed and was cleaned
skipping starting with img IMG_2539.jpg
skipping starting with img IMG_2673.jpg
skipping starting with img IMG_2564.jpg
skipping starting with img IMG_2541.jpg
skipping starting with img IMG_2559.jpg
skipping starting with img IMG_2580.jpg
skipping starting with img IMG_2667.jpg
skipping starting with img IMG_2610.jpg
skipping starting with img IMG_2530.jpg
skipping starting with img IMG_2671.jpg
skipping starting with img IMG_2576.jpg
skipping starting with img IMG_2560.jpg
skipping starting with img IMG_2563.jpg
skipping starting with img IMG_2663.jpg
skipping starting with img IMG_2582.jpg
skipping starting with img IMG_2668.jpg
skipping starting with img IMG_2612.jpg
skipping starting with img IMG_2664.jpg
skipping starting with img IMG_2616.jpg
skipping starting with img IMG_2531 2.jpg
skipping starting with img IMG_2540.jpg
skipping starting with img IMG_2570.jpg
skipping starting with img IMG_2618.jpg
skipping s